# Simulation of The Lost Boarding Puzzle using Monte Carlo
#### Iryna Didinova
100 people line up to board an airplane. Each has a boarding pass with assigned seat. However, the first person to board has lost his boarding pass and takes a random seat. After that, each person takes the assigned seat if it is unoccupied, and one of unoccupied seats at random otherwise. What is the probability that the last person to board gets to sit in his assigned seat?



In [11]:
rm(list = ls())
set.seed(123)

# Create a data frame for simulation results
results_airplane = data.frame(last_seats_in_assign_seat = 2:10001)

In [12]:
for( j in 1:10000){
    people = sample(1:100) # Passengers
    seats_random = sample(1:100) 
    seats_random_original = seats_random # Copy for check
    tickets = sample(1:100) 
    seats_taken = 0 # A column of places that passengers will eventually occupy 
  
    df = data.frame(people, tickets, seats_taken, seats_random,seats_random_original)
  
    df$tickets[1] = NA # The first passenger lost his ticket 
    df$seats_taken[1]=df$seats_random[1] # The first passenger chooses a seat at random 
    df$seats_random[1]=NA # No passenger will be able to occupy this seat 
  
    for(i in 2:99){ # Passengers 2 to 99 
    
    # Legal seat of the passenger i is NOT occupied  
        if(!(df$tickets[i] %in% df[1:(i-1),3])) {
        df$seats_taken[i] = df$tickets[i]
      # No passenger will be able to occupy this seat
        df$seats_random[which(df$seats_random == df$tickets[i])]= NA
      
    # Legal seat of the passenger i is occupied
        }else if (df$tickets[i] %in% df[1:(i-1),3]){
      # That's why they sit on a random free place - we choose a random number from the vector of 
        #random seats and then delete this number 
        df$seats_taken[i] = sample(na.omit(df$seats_random),1)
        df$seats_random[ which(df$seats_random == df$seats_taken[i])]= NA
        
        }else { # check
        print("CRITICAL ERROR !!!!!")
        }
    }
    

# If there is only 1 value left in the column "seats_random" "sample (na.omit (df $ seats_random), 1)
# does not select it but some random number from 1 to 100,
# therefore I will use "sum (na.omit (df $ seats_random))" for the last person 
  
  if(!(df$tickets[100] %in% df[1:100,3])) {
    
    df$seats_taken[100] = df$tickets[100]
    df$seats_random[which(df$seats_random == df$tickets[100])]= NA
    
  }else if (df$tickets[100] %in% df[1:100,3]){
    
    df$seats_taken[100] = sum(na.omit(df$seats_random))
    df$seats_random[ which(df$seats_random == df$seats_taken[100])]= NA
  }else {
    print("CRITICAL ERROR #1!!!!!")
  }
  
    
    
    ######################### Checks ########################
# These checks helped me notice the errors in the code and fix them 
  
  if (sum(duplicated(df$seats_taken)) >= 1){
    print("CRITICAL ERROR: seats taken are not unique!!!!!!!")
  } #else{ print("Check 1 : OK!!!")}
  
  if( sum(is.na(df$seats_taken))>= 1){
    print("CRITICAL ERROR: NA in seats taken!!!!!!!!")
  } #else{ print("Check 2 : OK!!!")}
  
  if(sum(df$seats_taken == 0) >=1){
    print("CRITICAL ERROR: seats taken is NULL!!!!!!!!")
  }#else{ print("Check 3 : OK!!!")}

    
    
    ### Fill in the results_airplane table with probability estimates  ###
  
  if(df$tickets[100] == df$seats_taken[100]){
    
    results_airplane[j,1] <- 1 # The last passenger sits in his assigned seat 
  } else {
    results_airplane[j,1] <- 0 # The last passenger does NO sit in his assigned seat 
  }
}

## Final calculation of the probability estimate 

In [13]:
mean(results_airplane$last_seats_in_assign_seat)  

[1] 0.4966

1000 observations: result = 0.494 \
10 000 observations: result = 0.4966 \
With an observation of 10,000, I got an estimate of the probability that is almost equal to the theoretical value of 0.5. 
